In [ ]:
from src.utils import petab as pet
import numpy as np
import src.utils.sbml as sbml
import src.utils.amici as am
from src.models.sbml_models import robertson_rxn, robertson_dae
%load_ext autoreload
%autoreload 2

In [ ]:
observables_df = pet.define_observables(
    {
        "A": "A",
        "B": "B",
        "C": "C",
    },
    noise_value=0.02,
)

# Load SBML/AMICI model
sbml_model_filepath = sbml.write_model(robertson_dae)

validator = sbml.validateSBML(ucheck=False)
validator.validate(sbml_model_filepath)

model = am.load_amici_model(
    sbml_model_filepath, observables_df=observables_df, force_compile=True
)

In [ ]:
# Define model parameters
true_params = {
    "k1": 0.04,
    "k2": 3.0e7,
    "k3": 1.0e4,
}
model = am.set_model_parameters(model, true_params)

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        "A": [1],
        "B": [0],
        "C": [0],
    }
)

# Define conversion values
timepoints = np.logspace(-5, 7, 400)

amici_measurements_df, rdatas = pet.define_measurements_amici(
    amici_model=model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True,
)

amici_measurements_df

In [ ]:
# Get measurement data for obs_A
obs_A_data = amici_measurements_df[amici_measurements_df["observableId"] == "obs_A"]
obs_B_data = amici_measurements_df[amici_measurements_df["observableId"] == "obs_B"]
obs_C_data = amici_measurements_df[amici_measurements_df["observableId"] == "obs_C"]

import matplotlib.pyplot as plt
plt.plot(obs_A_data["time"], obs_A_data["measurement"], label="A")
plt.plot(obs_B_data["time"], obs_B_data["measurement"]*1e4, label="B * 1e4")
plt.plot(obs_C_data["time"], obs_C_data["measurement"], label="C")
plt.xscale('log')
plt.ylabel("Concentration")
plt.xlabel("Time")
plt.legend()